In [2]:
import torch
torch.manual_seed(1337)

B,T,C = 4,8,2
x_b_t_c = torch.randn(B,T,C)
x_b_t_c.shape

xbow_b_t_c = torch.zeros(B,T,C) ## bow for bag of words, which means an average of tokens 
for b in range(B):
    for t in range(T):
        xprev_b_t_c = x_b_t_c[b,:t+1,:]
        xbow_b_t_c[b,t,:] = torch.mean(xprev_b_t_c,dim=0)

x_b_t_c[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [3]:

xbow_b_t_c[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [4]:
## compute xbow optimized 
def compute_xbow_opt(x_b_t_c):
    B,T,C = x_b_t_c.shape
    triu_upper_t_c = torch.triu(torch.ones(T, T), diagonal=0)
#    print(triu_upper_t_c.shape)
#    print(triu_upper_t_c.sum(dim=0,keepdim=True).shape)
#    print(triu_upper_t_c.sum(dim=1,keepdim=True).shape)
    
    xbow_b_t_c = torch.zeros(B,T,C) ## bow for bag of words, which means an average of tokens 

    for c in range(C):
        xbow_b_t_c[:,:,c] = x_b_t_c[:,:,c] @ triu_upper_t_c
#        xbow_b_t_c[:,:,c] /= torch.arange(1,T+1).view(1,-1)
        xbow_b_t_c[:,:,c] /= triu_upper_t_c.sum(dim=0, keepdim=True)
    return xbow_b_t_c

def compute_xbow_opt_v1(x_b_t_c):
    B,T,C = x_b_t_c.shape
    triu_down_t_c = torch.tril(torch.ones(T, T), diagonal=0)
    triu_down_t_c /= triu_down_t_c.sum(dim=1, keepdim=True)        
    return triu_down_t_c @ x_b_t_c

xbow_b_t_c = compute_xbow_opt(x_b_t_c)
print(xbow_b_t_c[0])

xbow_b_t_c_1 = compute_xbow_opt_v1(x_b_t_c)
print(xbow_b_t_c[0])

torch.allclose(xbow_b_t_c, xbow_b_t_c_1)    

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


True

In [7]:
import torch
import torch.nn as nn

torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # B x T x head_size
q = query(x) # B x T x head_size

wei  =q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T, T), diagonal=0)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = torch.softmax(wei, dim=-1)
print(wei[0])
v = value(x)
out = wei @ v 

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)
